In [2]:
import pandas as pd
import numpy as np

from datetime import timedelta
from datetime import datetime


In [ ]:
year = 2020
fecha = datetime.strptime(f'{year}-01-01', '%Y-%m-%d')
fecha_inicio = fecha - timedelta(days=96)
fecha_fin = datetime.strptime(f'{year+1}-01-01', '%Y-%m-%d')

In [21]:
data = pd.read_csv(f'data/{year}.csv')

In [15]:
data = data[data['Actividad_Realizada'] == 'Muestreo']
data['Etapa_Fasica'].value_counts()


Etapa_Fasica
Solitaria               840
Trasciens Congregans     43
Name: count, dtype: int64

In [16]:

data['Recomendaciones'].value_counts()


Recomendaciones
PEP6                                            49
PEP 06                                          16
SE APLICO CONTROL BIOLOGICO AEREO EN 50 HAS.     1
SE APLICO CONTROL BIOLOGICO AEREO EN 77 HAS      1
evaluacion de aplicacion con drone               1
área quemada                                     1
Àrea quemada                                     1
àrea con aplicaciòn quìmica                      1
evalucion de aplicacion con drone                1
Name: count, dtype: int64

In [6]:
data.drop(data[data['Resultado']=='En proceso'].index, inplace=True)
data['Resultado'].value_counts()

Resultado
Negativo    390
Positivo     18
Name: count, dtype: int64

In [ ]:
data_output = data[['Fecha','Latitud','Longitud','Resultado']]
data_output.loc[:,'Fecha_fin'] = data_output['Fecha'].strftime('%Y-%m-%d').astype('datetime64[s]')
data_output.loc[:,'Fecha_inicio'] = data_output['Fecha'] - timedelta(days=96)
data_output

In [8]:

def get_series(folder,field_name,row,names):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')  
    df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    list_f = df[names].values.tolist()
    len_list = len(list_f)  
    if len_list == 0:
        list_f = [0]*96
    if len_list != 96:
        #print(len(list_f))
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])
        #print(list_f)
        
        #print(len(list_f))
        #print(row)

    return list_f



def get_series_group(folder,field_name,row,names):
    df = pd.read_csv(folder +field_name+'/' +  str(row['Latitud']) + '_' + str(row['Longitud']) + '.csv')
    df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')  
    #df = df.set_index('date').asfreq('D', method='ffill').reset_index()
    mask = (df['date'] > row['Fecha_inicio']) & (df['date'] <= row['Fecha'])
    df = df.loc[mask]
    list_f = df[names].values.tolist()  
    len_list = len(list_f)  
    if len_list == 0:
        list_f = [0]*96
        len_list = len(list_f)  
    if len_list != 96:
        print('len list')
        print(len(list_f))
        for i in range(len_list,96):
            list_f.append(list_f[len_list-1])

        print('len list f')
        print(len(list_f))
        #print(row)

    return list_f

#get_series(data_output.iloc[0])x['NDVI'].tolist()




In [ ]:
print('NDVI')
data_output['NDVI'] = data_output.apply(
    lambda row: get_series(f'gee-coordinates-{year}/', \
    'NDVI',row,'NDVI'), axis=1)

print('LST_Day_1km')
data_output['LST_Day_1km'] = data_output.apply(
    lambda row: get_series(f'gee-coordinates-{year}/',
    'LST_Day_1km',row,'LST_Day_1km'), axis=1)

print('SOIL')
features2 =  ['Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst']
for feature in features2:
    data_output[feature] = data_output.apply(
        lambda row: get_series_group(f'gee-coordinates-{year}/', \
        'SOIL',row,feature), axis=1)

print('DAYMET')
features =  ['prcp','srad','tmax','tmin','vp']
for feature in features:
    data_output[feature] = data_output.apply(
            lambda row: get_series_group(f'gee-coordinates-{year}/',\
            'DAYMET_V4',row,feature), axis=1)

dataset = data_output[features+features2+['LST_Day_1km','NDVI','Resultado']]
dataset.to_csv('data/{year}-dataset.csv', index=False) 




In [ ]:

dataset = pd.read_csv('data/{year}-dataset.csv')

features_all = ['LST_Day_1km','NDVI', \
                'Psurf_f_inst','Qair_f_inst','Rainf_tavg','Tair_f_inst','Wind_f_inst','RootMoist_inst', \
                    'prcp','srad','tmax','tmin','vp']

def convert(row,feature,st,end):
    list_parsed = eval(row[feature])[st:end]
    if np.sum(list_parsed) == 0:
        return 0.0
    else:
        return np.mean(list_parsed)

feature_dataset = pd.DataFrame()
feature_dataset['Resultado'] = dataset['Resultado']
for feature in features_all:
    print(feature)
    feature_dataset[feature + '_1'] = dataset.apply(lambda row: convert(row,feature,0,12),axis=1)
    feature_dataset[feature + '_2'] = dataset.apply(lambda row: convert(row,feature,12,24),axis=1)
    feature_dataset[feature + '_3'] = dataset.apply(lambda row: convert(row,feature,24,36),axis=1)
    feature_dataset[feature + '_4'] = dataset.apply(lambda row: convert(row,feature,36,48),axis=1)
    feature_dataset[feature + '_5'] = dataset.apply(lambda row: convert(row,feature,48,60),axis=1)
    feature_dataset[feature + '_6'] = dataset.apply(lambda row: convert(row,feature,60,72),axis=1)
    feature_dataset[feature + '_7'] = dataset.apply(lambda row: convert(row,feature,72,84),axis=1)
    feature_dataset[feature + '_8'] = dataset.apply(lambda row: convert(row,feature,84,96),axis=1)

feature_dataset.to_csv('data/{year}-feature-dataset.csv', index=False) 
feature_dataset.describe()